In [ ]:
%%html
<h2>Keras によるコーディング</h2>
<ul>
  <li>drop 0.5</li>
  <li>activation: relu</li>
  <li>hidden layer: 3</li>
</ul>

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras import backend as K
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.random.seed(123)

'''
データの生成
'''
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

n = len(mnist.data)
N = 30000  # MNISTの一部を使う
N_train = 20000
N_validation = 4000
indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]

X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, train_size=N_train)
X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X_train, Y_train, test_size=N_validation)

'''
モデル設定
'''
n_in = len(X[0])  # 784
n_hiddens = [200, 200, 200]
n_out = len(Y[0])  # 10
p_keep = 0.5
activation = 'relu'


def weight_variable(shape):
    return K.truncated_normal(shape, stddev=0.01)
    # return np.random.normal(scale=0.01, size=shape)


model = Sequential()
for i, input_dim in enumerate(([n_in] + n_hiddens)[:-1]):
    model.add(Dense(n_hiddens[i], input_dim=input_dim,
                    kernel_initializer=weight_variable))
    model.add(Activation(activation))
    model.add(Dropout(p_keep))

model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

'''
モデル学習
'''
epochs = 50
batch_size = 200

hist = model.fit(X_train, Y_train, epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_validation, Y_validation))


'''
予測精度の評価
'''
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

Using TensorFlow backend.


Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 1s - loss: 2.2694 - acc: 0.1580 - val_loss: 2.1148 - val_acc: 0.3515
Epoch 2/50
16000/16000 [==============================] - 1s - loss: 1.4968 - acc: 0.4821 - val_loss: 0.7877 - val_acc: 0.7565
Epoch 3/50
16000/16000 [==============================] - 1s - loss: 0.8541 - acc: 0.7192 - val_loss: 0.5200 - val_acc: 0.8408
Epoch 4/50
16000/16000 [==============================] - 1s - loss: 0.6256 - acc: 0.8031 - val_loss: 0.3972 - val_acc: 0.8862
Epoch 5/50
16000/16000 [==============================] - 1s - loss: 0.5227 - acc: 0.8444 - val_loss: 0.3349 - val_acc: 0.9008
Epoch 6/50
16000/16000 [==============================] - 1s - loss: 0.4566 - acc: 0.8664 - val_loss: 0.2919 - val_acc: 0.9145
Epoch 7/50
16000/16000 [==============================] - 1s - loss: 0.3938 - acc: 0.8869 - val_loss: 0.2703 - val_acc: 0.9215
Epoch 8/50
16000/16000 [==============================] - 1s -

In [ ]:
'''
学習の進み具合を可視化
'''
val_acc = hist.history['val_acc']
val_loss = hist.history['val_loss']

plt.rc('font', family='serif')
fig = plt.figure()
plt.plot(range(epochs), val_acc, label='acc', color='black')
plt.xlabel('epochs')
plt.show()
